In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'


In [2]:
import os
import shutil
import pathlib
import gc
import pandas as pd
import ttach as tta
from pathlib import Path
from PIL import Image
import numpy as np
import cv2 as cv
import random
import matplotlib.pyplot as plt
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset

import torchvision
from torchvision import datasets
import cv2
from torch.cuda import amp

import torchvision.transforms as T
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.utils import make_grid
import optuna

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import timm

In [3]:
def seed_everything(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [23]:
seed_everything(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [5]:
class ContrailDataset:
    def __init__(self, df, transform=None):
        self.df = df  
        self.images = df['image']
        self.labels = df['label']
        self.transform =transform
        
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        image = np.load("../../input/" + self.images[idx]).astype(float)   
        label = np.load("../../input/" + self.labels[idx]).astype(float)
        
        # label_cls = 1 if label.sum() > 0 else 0
        if self.transform :
            data = self.transform(image=image, mask=label)
            image  = data['image']
            label  = data['mask']
            image = np.transpose(image, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1))    
            
        return torch.tensor(image), torch.tensor(label)

In [14]:
CFGS = [ 
    
    {
        'model_name': 'Unet',
        'backbone': 'resnet34',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '../../output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
        'threshold': 0.62, #0.24,
        'tta': True
    }
]

In [96]:
# https://www.kaggle.com/code/victorsd/2nd-place-inference

class ConvSilu(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(ConvSilu, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=1),
            nn.SiLU(inplace=True)
        )
    def forward(self, x):
        return self.layer(x)



# from coat import *


class Timm_Unet(nn.Module):
    def __init__(self, cfg, decoder_filters=[32, 48, 64, 96, 128], **kwargs):
        super(Timm_Unet, self).__init__()

        # backbone: 'resnet34'
        encoder = timm.create_model(
            cfg['backbone'],
            features_only=True,
            pretrained=True,
            in_chans=3, 
        )
        encoder_filters = [f['num_chs'] for f in encoder.feature_info]
        decoder_filters = decoder_filters
        
        print(encoder_filters, decoder_filters)
        

        self.conv6 = ConvSilu(encoder_filters[-1], decoder_filters[-1])
        self.conv6_2 = ConvSilu(decoder_filters[-1] + encoder_filters[-2], decoder_filters[-1])
        self.conv7 = ConvSilu(decoder_filters[-1], decoder_filters[-2])
        self.conv7_2 = ConvSilu(decoder_filters[-2] + encoder_filters[-3], decoder_filters[-2])
        self.conv8 = ConvSilu(decoder_filters[-2], decoder_filters[-3])
        self.conv8_2 = ConvSilu(decoder_filters[-3] + encoder_filters[-4], decoder_filters[-3])
        self.conv9 = ConvSilu(decoder_filters[-3], decoder_filters[-4])

        if len(encoder_filters) == 4:
            self.conv9_2 = None
        else:
            self.conv9_2 = ConvSilu(decoder_filters[-4] + encoder_filters[-5], decoder_filters[-4])
        
        self.conv10 = ConvSilu(decoder_filters[-4], decoder_filters[-5])
        
        self.res = nn.Conv2d(
            decoder_filters[-5],
            cfg['num_classes'], 
            1,
            stride=1,
            padding=0
        )

        self.cls = nn.Linear(encoder_filters[-1] * 2, 1)
        self.pix_sz =  nn.Linear(encoder_filters[-1] * 2, 1)
        self._initialize_weights()
        self.encoder = encoder


    def forward(self, x):
        batch_size, C, H, W = x.shape
        
        print(batch_size, C, H, W)

        if self.conv9_2 is None:
            enc2, enc3, enc4, enc5 = self.encoder(x)
        else:
            enc1, enc2, enc3, enc4, enc5 = self.encoder(x)

            
        print(enc1.shape, enc2.shape)
            
        dec6 = self.conv6(F.interpolate(enc5, scale_factor=2))
        dec6 = self.conv6_2(torch.cat([dec6, enc4
                ], 1))
        dec7 = self.conv7(F.interpolate(dec6, scale_factor=2))
        dec7 = self.conv7_2(torch.cat([dec7, enc3
                ], 1))
        
        dec8 = self.conv8(F.interpolate(dec7, scale_factor=2))
        dec8 = self.conv8_2(torch.cat([dec8, enc2
                ], 1))

        dec9 = self.conv9(F.interpolate(dec8, scale_factor=2))

        print(dec6.shape, dec7.shape, dec8.shape, dec9.shape )
        
        if self.conv9_2 is not None:
            dec9 = self.conv9_2(torch.cat([dec9, 
                    enc1
                    ], 1))
        
        dec10 = self.conv10(F.interpolate(dec9, scale_factor=2))
        
#         dec10 = self.conv10(dec9) # F.interpolate(dec9, scale_factor=2))
        print(dec10.shape)
        
        
        x1 = torch.cat([F.adaptive_avg_pool2d(enc5, output_size=1).view(batch_size, -1), 
                        F.adaptive_max_pool2d(enc5, output_size=1).view(batch_size, -1)], 1)

        # x1 = F.dropout(x1, p=0.3, training=self.training)
        organ_cls = self.cls(x1)
        pixel_size = self.pix_sz(x1)

        return self.res(dec10), organ_cls, pixel_size


    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Linear):
                m.weight.data = nn.init.kaiming_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

In [97]:
decoder_filters= [16, 32, 64, 256, 512] #[32, 48, 64, 96, 128, 256]
m = Timm_Unet(CFGS[0], decoder_filters=decoder_filters)
m.to(device)
print()

[64, 64, 128, 256, 512] [16, 32, 64, 256, 512]



In [98]:
# m.cuda()

In [99]:
im = torch.ones([8, 3,256,256])
im  = im.to(device, dtype=torch.float)
preds, cls, pix = m(im)
preds.shape

8 3 256 256
torch.Size([8, 64, 128, 128]) torch.Size([8, 64, 64, 64])
torch.Size([8, 512, 16, 16]) torch.Size([8, 256, 32, 32]) torch.Size([8, 64, 64, 64]) torch.Size([8, 32, 128, 128])
torch.Size([8, 16, 256, 256])


torch.Size([8, 1, 256, 256])

In [193]:
###TImmunetplusplus model Nirjhar

n_blocks = 4

class TimmSegModel(nn.Module):
    def __init__(self, backbone, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            backbone,
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(
            decoder_channels[n_blocks-1],
            cfg["num_classes"], 
            kernel_size=(3, 3),
            stride=(1, 1), 
            padding=(1, 1)
        )

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    

#path='./exp/baselinev2/qishentrialv2_tf_efficientnet_b7_ns_last_epoch-00.bin'

def build_model_timmunetplus(backbone, cfg):
    model = TimmSegModel(backbone, cfg)
    return model

def load_model_timmunetplus(path, backbone, cfg):
    model = build_model_timmunetplus(backbone, cfg)
    model.load_state_dict(torch.load(path))
    return model


#### Model 2 Nirjhar

n_blocks =4
class TimmSegModel2(nn.Module):
    def __init__(self, backbone, segtype='unet', pretrained=True):
        super(TimmSegModel2, self).__init__()

        self.encoder = timm.create_model(
            backbone,
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 128, 128))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
#path='./exp/baselinev2/qishentrialv2_tf_efficientnet_b7_ns_last_epoch-00.bin'


def build_model(backbone):
    model = TimmSegModel2(backbone, segtype='unet')
    return model

def load_model(path,backbone):
    model = build_model(backbone)
    model.load_state_dict(torch.load(path))
    return model




## Rohit Model 
class Net(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        self.model = smp.Unet(
            encoder_name=cfg["backbone"],     
            encoder_weights=None,   
            in_channels=3,  
            classes=cfg["num_classes"],
            activation=None
        )
        
    
    def forward(self, inputs):
        mask = self.model(inputs)
        return mask

In [194]:
def dice_coef(y_true, y_pred, thr=0.5, epsilon=1e-6):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum()
    den = y_true.sum() + y_pred.sum()
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    
    return dice


def get_transform(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_NEAREST),
    ], p=1.0)
    return transform

In [195]:
val_df =pd.read_csv("../../input/val_df_filled.csv")
# val_df = val_df.loc[val_df['class'] == 1].reset_index(drop=True)
# val_df = val_df.loc[val_df['class'] == 0].reset_index(drop=True)


final_preds = []

for idx, cfg in enumerate(CFGS):
    print(cfg)
    val_transform = get_transform(cfg['img_size'])
    valid_dataset = ContrailDataset(val_df, transform=val_transform)  

    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = 32, #32, 
        shuffle = False, 
        num_workers = 2, 
        pin_memory = True, 
        drop_last = False
    )

    


    if ensemple_type == "mine":
        model = Net(cfg)    
        model = torch.nn.DataParallel(model).cuda()
        model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    else:
        if idx == 0:
            model = load_model_timmunetplus(cfg['model_pth'], cfg['backbone'], cfg)
        elif idx == 1:
            model = load_model(cfg['model_pth'], cfg['backbone'])

        else:
            model = Net(cfg)    
            model = torch.nn.DataParallel(model).cuda()
            model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])

    
        
    
    if cfg['tta']:
        if ensemple_type == "mine":
            model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')
        else:
            if idx == 0 or idx == 1:
                model = tta.SegmentationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')
            else:
                model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')

    
    
    
    
    model.to(device)
    model.eval()
    
    preds = []
    masks_ = []
    
    for index, (images, masks) in enumerate(tqdm(valid_loader)):  
        images  = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        if cfg['img_size'][0] != 256:
            masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest') 
            
        masks_.append(torch.squeeze(masks, dim=1))

        with torch.inference_mode():
            images = torch.nn.functional.interpolate(images,size=cfg['img_size'][0], mode='nearest')
            pred = model(images)                     
            pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest') 

            preds.append(torch.squeeze(pred, dim=1))

    model_masks = torch.cat(masks_, dim=0)
    model_preds = torch.cat(preds, dim=0)
    
    model_masks = torch.flatten(model_masks, start_dim=0, end_dim=1)
    model_preds = torch.flatten(model_preds, start_dim=0, end_dim=1)  
    
    
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, model_preds.sigmoid(), thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    
        
    print(best_dice_score, best_threshold)
    final_preds.append(model_preds)
    
    del model
    torch.cuda.empty_cache()
    gc.collect()

{'model_name': 'Unet++', 'backbone': 'tf_efficientnet_b7_ns', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '../../output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin', 'threshold': 0.62, 'tta': True}


  0%|          | 0/58 [00:00<?, ?it/s]

0.6571967 0.32
{'model_name': 'Unet', 'backbone': 'eca_nfnet_l1', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '../../output/nirjhar/unetplusecarnf01_eca_nfnet_l1_best_epochstage2cv656-00.bin', 'threshold': 0.62, 'tta': True}


  0%|          | 0/58 [00:00<?, ?it/s]

0.658865 0.11
{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '../../output/exp_s1_pl_03/pl_round1/efficientnet-b7-256-s2/checkpoint_dice.pth', 'threshold': 0.24, 'tta': True}


...

0.67871666 0.15
{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '../../output/exp_pre/Unet/efficientnet-b7-256-s3/checkpoint_dice.pth', 'threshold': 0.24, 'tta': True}


  0%|          | 0/58 [00:00<?, ?it/s]

0.67894846 0.17
{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/contrail/output/exp_s1_pl_03/pl_round2/efficientnet-b7-256-s2/checkpoint_dice.pth', 'threshold': 0.62, 'tta': True}


  0%|          | 0/58 [00:00<?, ?it/s]

0.67928106 0.49


In [ ]:
# score = dice_coef(model_masks, final_preds[0], thr=0.5).cpu().detach().numpy() # get_score(labels, preds) 
# score

# 0.65458226 0.83
# 0.6572782 0.1

In [199]:
def boost_dice(params):    
    for index, val in enumerate(params.keys()):
        if index == 0:
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    
    
#     threshold = 0.5
# #     preds1 = (nn.Sigmoid()(preds)>threshold).double()     
#     preds1 = (nn.Sigmoid()(preds)>threshold).double()            
#     score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 

    best_threshold = 0.0
    best_score = 0.0
    for threshold in [i / 100 for i in range(101)] :    
        preds1 = (nn.Sigmoid()(preds)>threshold).double()            
        score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
        if score > best_score:
            best_score = score
            best_threshold = threshold
            
#     best_score = score
#     best_threshold = threshold
    return best_score, best_threshold

In [200]:
def objective(trial):
    params = {}    
    for i in range(len(final_preds)):
        params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1) 
        
#         params[f"w1"] = 0.5 #trial.suggest_float(f'w1', 0, 1) 
#         params[f"w2"] = 0.5 #trial.suggest_float(f'w2', 0, 1) 
#         params[f"w3"] = 0.5 # trial.suggest_float(f'w3', 0, 1) 
# #         params[f"w4"] = 0.25 #trial.suggest_float(f'w4', 0, 1) 
    
        
    score, best_threshold  = boost_dice(params)
    params['threshold'] = best_threshold
    print(params)
    return score

study = optuna.create_study(direction='maximize')
# study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=1500)

best_params = study.best_params
best_params

[I 2023-06-15 15:30:54,231] A new study created in memory with name: no-name-8a52890f-deba-4ba4-bb1c-2eca94656c3b
[I 2023-06-15 15:30:55,263] Trial 0 finished with value: 0.6815356016159058 and parameters: {'w1': 0.856824259521216, 'w2': 0.955512191950857, 'w3': 0.014405019911556849, 'w4': 0.9926600657207473, 'w5': 0.028744586511597725, 'w6': 0.768397374636816}. Best is trial 0 with value: 0.6815356016159058.


{'w1': 0.856824259521216, 'w2': 0.955512191950857, 'w3': 0.014405019911556849, 'w4': 0.9926600657207473, 'w5': 0.028744586511597725, 'w6': 0.768397374636816, 'threshold': 0.11}


[I 2023-06-15 15:30:56,272] Trial 1 finished with value: 0.6845129132270813 and parameters: {'w1': 0.2812687212646974, 'w2': 0.4147513953198575, 'w3': 0.3165077576398553, 'w4': 0.04215026787115772, 'w5': 0.6672528585781473, 'w6': 0.5033892428691097}. Best is trial 1 with value: 0.6845129132270813.


{'w1': 0.2812687212646974, 'w2': 0.4147513953198575, 'w3': 0.3165077576398553, 'w4': 0.04215026787115772, 'w5': 0.6672528585781473, 'w6': 0.5033892428691097, 'threshold': 0.15}


[I 2023-06-15 15:30:57,282] Trial 2 finished with value: 0.6838257312774658 and parameters: {'w1': 0.13594746889503562, 'w2': 0.3778410625473666, 'w3': 0.5269930793516882, 'w4': 0.381525182000097, 'w5': 0.28941110618817456, 'w6': 0.8024294390495573}. Best is trial 1 with value: 0.6845129132270813.


...

[I 2023-06-15 15:57:36,594] Trial 1498 finished with value: 0.6837189793586731 and parameters: {'w1': 0.21128920977284213, 'w2': 0.4879546882713428, 'w3': 0.9745054679504273, 'w4': 0.34099342414933287, 'w5': 0.6597072111994106, 'w6': 0.6094160061472534}. Best is trial 1286 with value: 0.6850342154502869.


{'w1': 0.21128920977284213, 'w2': 0.4879546882713428, 'w3': 0.9745054679504273, 'w4': 0.34099342414933287, 'w5': 0.6597072111994106, 'w6': 0.6094160061472534, 'threshold': 0.12}


[I 2023-06-15 15:57:37,687] Trial 1499 finished with value: 0.6838476657867432 and parameters: {'w1': 0.1944342962818746, 'w2': 0.42127203232655447, 'w3': 0.0329750110475021, 'w4': 0.13020108903952432, 'w5': 0.5956972206026637, 'w6': 0.38166279565328876}. Best is trial 1286 with value: 0.6850342154502869.


{'w1': 0.1944342962818746, 'w2': 0.42127203232655447, 'w3': 0.0329750110475021, 'w4': 0.13020108903952432, 'w5': 0.5956972206026637, 'w6': 0.38166279565328876, 'threshold': 0.17}


{'w1': 0.1976000599839957,
 'w2': 0.4126970697935236,
 'w3': 0.0010400328806771247,
 'w4': 0.13358923329787326,
 'w5': 0.6293367385421876,
 'w6': 0.8157294847307331}

In [122]:


# 0.6835659742355347 and parameters: {'w1': 0.012511663696378001, 'w2': 0.7308678705489249, 'w3': 0.7309363859834449, 'w4': 0.8998292052691779, 'w5': 0.9377896156028775}. Best is trial 183 with value: 0.6835659742355347.
# TH: 0.1

# pl3_round2_s2 included at last
# Trial 177 finished with value: 0.6850342154502869 and parameters: 
#{'w1': 0.1976000599839957, 'w2': 0.4126970697935236, 'w3': 0.0010400328806771247, 'w4': 0.13358923329787326, 'w5': 0.6293367385421876, 'w6': 0.8157294847307331}. Best is trial 1286 with value: 0.6850342154502869.
#TH:0.19


# 0.6850342154502869
# {'w1': 0.1976000599839957,
#  'w2': 0.4126970697935236,
#  'w3': 0.0010400328806771247,
#  'w4': 0.13358923329787326,
#  'w5': 0.6293367385421876,
#  'w6': 0.8157294847307331}







#th: 0.2
# {'w1': 0.9118427143017789, 'w2': 0.07722806128805348, 'w3': 0.3483576258941602}


# 0.6802493929862976
# {'w1': 0.901456237984701,
#  'w2': 0.03333500127663481,
#  'w3': 0.1012744451767772,
#  'w4': 0.3315684160625839}


# 0.6802645325660706
# {'w1': 0.9423043059931125,
#  'w2': 0.12999481950972483,
#  'w3': 0.34096541343299724}




# 0.6753096580505371
# {'w1': 0.8191544364616984,
#  'w2': 0.2172873170900641,
#  'w3': 0.9872223348463418,
#  'w4': 0.34763898831692425,
#  'w5': 0.6106201134139329}


In [ ]:
best_params

In [ ]:
# {'w1': 0.19193466652838265, 'w2': 0.0010692529855302647, 'w3': 0.19479657217496682, 'w4': 0.7062431810854415, 'threshold': 0.39}


In [ ]:
#  0.66671222448349 and parameters:
# {'w1': 0.7800660379809229, 'w2': 0.5851307264271756, 'w3': 0.5728701966485974}
# threshold: 0.32


In [204]:
import pretrainedmodels

def conv3x3(in_channel, out_channel): #not change resolusion
    return nn.Conv2d(in_channel,out_channel,
                      kernel_size=3,stride=1,padding=1,dilation=1,bias=False)

def conv1x1(in_channel, out_channel): #not change resolution
    return nn.Conv2d(in_channel,out_channel,
                      kernel_size=1,stride=1,padding=0,dilation=1,bias=False)

def init_weight(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        #nn.init.xavier_uniform_(m.weight, gain=1)
        #nn.init.xavier_normal_(m.weight, gain=1)
        #nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
        nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
        #nn.init.orthogonal_(m.weight, gain=1)
        if m.bias is not None:
            m.bias.data.zero_()
    elif classname.find('Batch') != -1:
        m.weight.data.normal_(1,0.02)
        m.bias.data.zero_()
    elif classname.find('Linear') != -1:
        nn.init.orthogonal_(m.weight, gain=1)
        if m.bias is not None:
            m.bias.data.zero_()
    elif classname.find('Embedding') != -1:
        nn.init.orthogonal_(m.weight, gain=1)
    

class ChannelAttentionModule(nn.Module):
    def __init__(self, in_channel, reduction):
        super().__init__()
        self.global_maxpool = nn.AdaptiveMaxPool2d(1)
        self.global_avgpool = nn.AdaptiveAvgPool2d(1) 
        self.fc = nn.Sequential(
            conv1x1(in_channel, in_channel//reduction).apply(init_weight),
            nn.ReLU(True),
            conv1x1(in_channel//reduction, in_channel).apply(init_weight)
        )
        
    def forward(self, inputs):
        x1 = self.global_maxpool(inputs)
        x2 = self.global_avgpool(inputs)
        x1 = self.fc(x1)
        x2 = self.fc(x2)
        x  = torch.sigmoid(x1 + x2)
        return x
    
    
class SpatialAttentionModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv3x3 = conv3x3(2,1).apply(init_weight)
        
    def forward(self, inputs):
        x1,_ = torch.max(inputs, dim=1, keepdim=True)
        x2 = torch.mean(inputs, dim=1, keepdim=True)
        x  = torch.cat([x1,x2], dim=1)
        x  = self.conv3x3(x)
        x  = torch.sigmoid(x)
        return x
    
    
class CBAM(nn.Module):
    def __init__(self, in_channel, reduction):
        super().__init__()
        self.channel_attention = ChannelAttentionModule(in_channel, reduction)
        self.spatial_attention = SpatialAttentionModule()
        
    def forward(self, inputs):
        x = inputs * self.channel_attention(inputs)
        x = x * self.spatial_attention(x)
        return x
    
    
class CenterBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        self.conv = conv3x3(in_channel, out_channel).apply(init_weight)
        
    def forward(self, inputs):
        x = self.conv(inputs)
        return x


class DecodeBlock(nn.Module):
    def __init__(self, in_channel, out_channel, upsample):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_channel).apply(init_weight)
        self.upsample = nn.Sequential()
        if upsample:
            self.upsample.add_module('upsample',nn.Upsample(scale_factor=2, mode='nearest'))
        self.conv3x3_1 = conv3x3(in_channel, in_channel).apply(init_weight)
        self.bn2 = nn.BatchNorm2d(in_channel).apply(init_weight)
        self.conv3x3_2 = conv3x3(in_channel, out_channel).apply(init_weight)
        self.cbam = CBAM(out_channel, reduction=16)
        self.conv1x1   = conv1x1(in_channel, out_channel).apply(init_weight)
        
    def forward(self, inputs):
        x  = F.relu(self.bn1(inputs))
        x  = self.upsample(x)
        x  = self.conv3x3_1(x)
        x  = self.conv3x3_2(F.relu(self.bn2(x)))
        x  = self.cbam(x)
        x += self.conv1x1(self.upsample(inputs)) #shortcut
        return x
 
            
#U-Net SeResNext101 + CBAM + hypercolumns + deepsupervision
class UNET_SERESNEXT101(nn.Module):
    def __init__(self, resolution, deepsupervision, clfhead, clf_threshold, load_weights=True):
        super().__init__()
        h,w = resolution
        self.deepsupervision = deepsupervision
        self.clfhead = clfhead
        self.clf_threshold = clf_threshold
        
        #encoder
        model_name = 'se_resnext101_32x4d'
        if load_weights:
            seresnext101 = pretrainedmodels.__dict__[model_name](pretrained='imagenet')
        else:
            seresnext101 = pretrainedmodels.__dict__[model_name](pretrained=None)
        
        self.encoder0 = nn.Sequential(
            seresnext101.layer0.conv1, #(*,3,h,w)->(*,64,h/2,w/2)
            seresnext101.layer0.bn1,
            seresnext101.layer0.relu1,
        )
        self.encoder1 = nn.Sequential(
            seresnext101.layer0.pool, #->(*,64,h/4,w/4)
            seresnext101.layer1 #->(*,256,h/4,w/4)
        )
        self.encoder2 = seresnext101.layer2 #->(*,512,h/8,w/8)
        self.encoder3 = seresnext101.layer3 #->(*,1024,h/16,w/16)
        self.encoder4 = seresnext101.layer4 #->(*,2048,h/32,w/32)
        
        #center
        self.center  = CenterBlock(2048,512) #->(*,512,h/32,w/32)
        
        #decoder
        self.decoder4 = DecodeBlock(512+2048,64, upsample=True) #->(*,64,h/16,w/16)
        self.decoder3 = DecodeBlock(64+1024,64, upsample=True) #->(*,64,h/8,w/8)
        self.decoder2 = DecodeBlock(64+512,64,  upsample=True) #->(*,64,h/4,w/4) 
        self.decoder1 = DecodeBlock(64+256,64,   upsample=True) #->(*,64,h/2,w/2) 
        self.decoder0 = DecodeBlock(64,64, upsample=True) #->(*,64,h,w) 
        
        #upsample
        self.upsample4 = nn.Upsample(scale_factor=16, mode='bilinear', align_corners=True)
        self.upsample3 = nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True)
        self.upsample2 = nn.Upsample(scale_factor=4, mode='bilinear', align_corners=True)
        self.upsample1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        #deep supervision
        self.deep4 = conv1x1(64,1).apply(init_weight)
        self.deep3 = conv1x1(64,1).apply(init_weight)
        self.deep2 = conv1x1(64,1).apply(init_weight)
        self.deep1 = conv1x1(64,1).apply(init_weight)
        
        #final conv
        self.final_conv = nn.Sequential(
            conv3x3(320,64).apply(init_weight),
            nn.ELU(True),
            conv1x1(64,1).apply(init_weight)
        )
        
        #clf head
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.clf = nn.Sequential(
            nn.BatchNorm1d(2048).apply(init_weight),
            nn.Linear(2048,512).apply(init_weight),
            nn.ELU(True),
            nn.BatchNorm1d(512).apply(init_weight),
            nn.Linear(512,1).apply(init_weight)
        )
        
        
    def forward(self, inputs):
        #encoder
        x0 = self.encoder0(inputs) #->(*,64,h/2,w/2)
        x1 = self.encoder1(x0) #->(*,256,h/4,w/4)
        x2 = self.encoder2(x1) #->(*,512,h/8,w/8)
        x3 = self.encoder3(x2) #->(*,1024,h/16,w/16)
        x4 = self.encoder4(x3) #->(*,2048,h/32,w/32)
        
        #clf head
        logits_clf = self.clf(self.avgpool(x4).squeeze(-1).squeeze(-1)) #->(*,1)
        if (not self.training) & (self.clf_threshold is not None):
            if (torch.sigmoid(logits_clf)>self.clf_threshold).sum().item()==0:
                bs,_,h,w = inputs.shape
                logits = torch.zeros((bs,1,h,w))
                if self.clfhead:
                    if self.deepsupervision:
                        return logits,_,_
                    else:
                        return logits,_
                else:
                    if self.deepsupervision:
                        return logits,_
                    else:
                        return logits
        
        #center
        y5 = self.center(x4) #->(*,320,h/32,w/32)
        
        #decoder
        y4 = self.decoder4(torch.cat([x4,y5], dim=1)) #->(*,64,h/16,w/16)
        y3 = self.decoder3(torch.cat([x3,y4], dim=1)) #->(*,64,h/8,w/8)
        y2 = self.decoder2(torch.cat([x2,y3], dim=1)) #->(*,64,h/4,w/4)
        y1 = self.decoder1(torch.cat([x1,y2], dim=1)) #->(*,64,h/2,w/2) 
        y0 = self.decoder0(y1) #->(*,64,h,w)
        
        #hypercolumns
        y4 = self.upsample4(y4) #->(*,64,h,w)
        y3 = self.upsample3(y3) #->(*,64,h,w)
        y2 = self.upsample2(y2) #->(*,64,h,w)
        y1 = self.upsample1(y1) #->(*,64,h,w)
        hypercol = torch.cat([y0,y1,y2,y3,y4], dim=1)
        
        #final conv
        logits = self.final_conv(hypercol) #->(*,1,h,w)
        
        if self.clfhead:
            if self.deepsupervision:
                s4 = self.deep4(y4)
                s3 = self.deep3(y3)
                s2 = self.deep2(y2)
                s1 = self.deep1(y1)
                logits_deeps = [s4,s3,s2,s1]
                return logits, logits_deeps, logits_clf
            else:
                return logits, logits_clf
        else:
            if self.deepsupervision:
                s4 = self.deep4(y4)
                s3 = self.deep3(y3)
                s2 = self.deep2(y2)
                s1 = self.deep1(y1)
                logits_deeps = [s4,s3,s2,s1]
                return logits, logits_deeps
            else:
                return logits
            
            


In [205]:
def build_model(model_name, resolution, deepsupervision, clfhead, clf_threshold, load_weights):
    if model_name=='seresnext101':
        model = UNET_SERESNEXT101(resolution, deepsupervision, clfhead, clf_threshold, load_weights)
    return model

In [219]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


model = build_model("seresnext101", (256,256), False, False, 0.5, load_weights=True )
# model.cuda()


In [220]:
# model()

In [222]:
a = torch.ones(64, 3, 256, 256)
out = model(a)

In [224]:
out.shape

torch.Size([64, 1, 256, 256])